In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.dots_and_boxes.NNmodels.transformer import TransformerInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'transformer'
model_params: TransformerInitParams = {
    'num_rows': 3,
    'num_cols': 2,
    'embed_dim': 128,
    'feedforward_dim': 512,
    'num_heads': 4,
    'attention_layers': 4
}
device = torch.device('mps')
model_name = 'dots_and_boxes_transformer'

## Initialize new model
load_model = None
load_model_params = {}

## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 10,
    'cooldown': 10,
    'min_lr': 1e-6
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 100,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load replay buffer from wandb
load_replay_buffer = 'from_file'
load_replay_buffer_params = {
    'path': f'applications/dots_and_boxes/training_data/dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_DABSimpleTensorMapping_minimax.pkl',
    'device': device
}
# load_replay_buffer = 'from_wandb'
# load_replay_buffer_params = {
#     'project': 'AlphaZero-DotsAndBoxes',
#     'artifact_name': f'dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_SimpleTensorMapping_minimax',
#     'artifact_version': 'latest'
# }


In [3]:
# Initialize wandb run
import wandb

run_name = 'Transformer 3'
notes = 'New transformer model, supervised training on 3 x 2 board. Comparable parameters to Linear Attention Transformer 3.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-DotsAndBoxes',
    name=run_name,
    config=config,
    notes=notes,
    group=f'{training_method} training on {model_params["num_rows"]}x{model_params["num_cols"]} board'
)
# run = None

wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
# Perform training

from applications.dots_and_boxes.train import train

model_interface = train(
    num_rows = model_params["num_rows"],
    num_cols = model_params["num_cols"],
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

TypeError: DotsAndBoxesTransformer.__init__() got an unexpected keyword argument 'ff_dim'

In [5]:
run.finish()

Minimax_draw_rate,█████████████▃▂████████▇██▁█████████████
Minimax_loss_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▁▁▁▁▁▁▁▁▂▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimax_score,▅▅▅▅▅▅▅▅▅▅▅▅▅▆█▅▅▅▅▅▅▅▅▅▅▅▁▅▅▅▅▅▅▅▅▅▅▅▅▅
Minimax_win_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
RandomAgent_draw_rate,▅▄█▇▄▅▃▄▂▄▄▄▂▂▄▄▅▄▄▅▇▅▃▃▅▃▅▃▃▁▃▃▃▄▃▃▃▃▂▂
RandomAgent_loss_rate,▃▆▃▃▆▆▂▂▃▂▃▂▂▃▁▃█▃▁▃▁▃▂▃▃▂▂▄▁▂▃▂▃▃▂▃▁▂▃▃
RandomAgent_score,▄▃▁▂▃▂▇▆▇▆▅▅▇▆▆▄▁▄▆▄▃▃▆▅▄▆▅▅▇█▆▆▅▅▆▆▇▇▇▆
RandomAgent_win_rate,▄▄▁▂▄▃▆▅▇▅▅▅▇▇▅▅▂▅▆▄▃▃▆▅▄▆▄▅▆█▆▆▆▅▆▆▇▆▇▆
epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇███
learning_rate,███████████████▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▂▂▂▂▁
train_loss,█▇▇▇▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 13,602
Trainable parameters: 13,602 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
pos_embedding: 384 parameters
input_embedding.weight: 32 parameters
transformer_blocks.0.norm1.weight: 32 parameters
transformer_blocks.0.norm1.bias: 32 parameters
transformer_blocks.0.attn.q_emb.weight: 1,024 parameters
transformer_blocks.0.attn.q_emb.bias: 32 parameters
transformer_blocks.0.attn.k_emb.weight: 1,024 parameters
transformer_blocks.0.attn.k_emb.bias: 32 parameters
transformer_blocks.0.attn.v_emb.weight: 1,024 parameters
transformer_blocks.0.attn.v_emb.bias: 32 parameters
transformer_blocks.0.attn.out_emb.weight: 1,024 parameters
transformer_blocks.0.attn.out_emb.bias: 32 parameters
transformer_blocks.0.norm2.weight: 32 parameters
transformer_blocks.0.norm2.bias: 32 parameters
transformer_blocks.0.ff.0.weight: 4,096 parameters
transformer_blocks.0.ff.0.bias: 128 parameters
transformer_blocks.0.ff.2.weight: 4,096 parameters
transformer_blocks.0.ff.2.bia